# Steady State Solve

Example is based on Julian's Laplace example.

In [1]:
import petsc4py.PETSc as PETSc
import underworld3 as uw

In [2]:
options = PETSc.Options()

In [3]:
options.setValue("dm_plex_separate_marker", None)
options.setValue("temperature_petscspace_degree", 1)
options.setValue("sol_vec_view", None)
options.setValue("dm_view", None)
options.setValue("options_view",None)

In [4]:
user = {"y0": -0.6,
        "y1": 1.3,
        "k": 0.5,
        "h": 10,
        "T0": 4.0,
        "T1": 8.0,
        "simplex": True}

In [5]:
snes = PETSc.SNES().create(comm=PETSc.COMM_WORLD)

## Create Mesh

In [6]:
plex = PETSc.DMPlex().createBoxMesh(faces=(10,10), 
                                    lower=(-1.0, user["y0"]),
                                    upper=(1.0, user["y1"]),
                                    simplex=user["simplex"])

In [7]:
plex.distribute()

In [8]:
plex.setFromOptions()

In [9]:
plex.view()

In [10]:
part = plex.getPartitioner()
part.setFromOptions()
plex.distribute()

In [11]:
plex.localizeCoordinates()
plex.setFromOptions()

## Setup Discretisation

In [12]:
from underworld3.poisson import pySetupDiscretization

In [13]:
pySetupDiscretization(plex, user)

In [14]:
plex.view()

In [15]:
plex.createClosureIndex(None)
plex.setSNESLocalFEM()
u = plex.createGlobalVector()
snes.setDM(plex)
#snes.setFromOptions()

In [16]:
#import numpy as np
#u[:] = np.ones_like(u.array) * 0.0

In [17]:
#u.view()

In [18]:
#u.setName("Initial Solution")
u.setName("Solution")

In [19]:
snes.solve(None, u)

In [20]:
u.view()

In [21]:
viewer = PETSc.Viewer().createHDF5("temperature.h5", "w")
viewer(u)
viewer(plex)
del viewer
from underworld3.petsc_gen_xdmf import generateXdmf
generateXdmf("temperature.h5")